# Quantitative Value Strategy
"Value investing" means investing in the stocks that are cheapest relative to common measures of business value (like earnings or assets).

For this project, we're going to build an investing strategy that selects the 50 stocks with the best value metrics. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.

## Library Imports
Importing the open-source software libraries that we'll be using.

In [1]:
import numpy as np
import pandas as pd
import xlsxwriter
import requests
from scipy import stats
import math
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

## Importing Our List of Stocks & API Token
We need to import our list of stocks and our API token before proceeding

In [2]:
stocks = pd.read_csv('sp_500_stocks.csv')
from secrets import IEX_CLOUD_API_TOKEN

## Making Our First API Call

Start by building a simple value screener that ranks securities based on a single metric (the price-to-earnings ratio).

In [3]:
symbol = 'AAPL'
api_url = f'https://cloud.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data

{'avgTotalVolume': 54494501,
 'calculationPrice': 'tops',
 'change': -3.62,
 'changePercent': -0.01989,
 'close': None,
 'closeSource': 'official',
 'closeTime': None,
 'companyName': 'Apple Inc',
 'currency': 'USD',
 'delayedPrice': None,
 'delayedPriceTime': None,
 'extendedChange': None,
 'extendedChangePercent': None,
 'extendedPrice': None,
 'extendedPriceTime': None,
 'high': None,
 'highSource': None,
 'highTime': None,
 'iexAskPrice': 178.38,
 'iexAskSize': 100,
 'iexBidPrice': 177.8,
 'iexBidSize': 115,
 'iexClose': 178.37,
 'iexCloseTime': 1691425758282,
 'iexLastUpdated': 1691425761960,
 'iexMarketPercent': 0.025864871957959105,
 'iexOpen': 182.17,
 'iexOpenTime': 1691415000134,
 'iexRealtimePrice': 178.37,
 'iexRealtimeSize': 100,
 'iexVolume': 1500677,
 'lastTradeTime': 1691425760960,
 'latestPrice': 178.37,
 'latestSource': 'IEX real time price',
 'latestTime': '12:29:21 PM',
 'latestUpdate': 1691425761960,
 'latestVolume': None,
 'low': None,
 'lowSource': None,
 'lowTim

## Parsing Our API Call
This API call has the metric we need - the price-to-earnings ratio.

Here is an example of how to parse the metric from our API call:

In [4]:
price = data['latestPrice']
pe_ratio = data['peRatio']

## Executing A Batch API Call & Building Our DataFrame



In [5]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'Price-to-Earnings Ratio', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [6]:
final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://cloud.iexapis.com/stable/stock/market/batch/?types=quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    
    for symbol in symbol_string.split(','):
        if symbol in data:
            final_dataframe = final_dataframe.append(
            pd.Series(
            [
                symbol,
                data[symbol]['quote']['latestPrice'],
                data[symbol]['quote']['peRatio'],
                'N/A'
            ],
                index = my_columns
            ),
                ignore_index = True
            )
        else:
            print(f'Symbol not found: {symbol}')

final_dataframe

Symbol not found: DISCA
Symbol not found: HFC
Symbol not found: VIAC
Symbol not found: WLTW


,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,A,126.44,None,N/A
1,AAL,15.8,3.68,N/A
2,AAP,70.66,10.5,N/A
3,AAPL,178.35,30.33,N/A
4,ABBV,150.17,35.42,N/A
...,...,...,...,...
496,YUM,135.7,31.85,N/A
497,ZBH,125.87,52.45,N/A
498,ZBRA,247.085,19.78,N/A
499,ZION,38.17,3.91,N/A


## Removing Glamour Stocks

The opposite of a "value stock" is a "glamour stock". 

Since the goal of this strategy is to identify the 50 best value stocks from our universe, our next step is to remove glamour stocks from the DataFrame.

We'll sort the DataFrame by the stocks' price-to-earnings ratio, and drop all stocks outside the top 50.

In [7]:
final_dataframe.sort_values('Price-to-Earnings Ratio', inplace = True)
final_dataframe = final_dataframe[final_dataframe['Price-to-Earnings Ratio'] > 0]
final_dataframe = final_dataframe[:51]
final_dataframe.reset_index(inplace = True)
final_dataframe.drop('index', axis=1, inplace = True)
final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,BRK.B,363.24,0.07,N/A
1,FRC,3.51,0.43,N/A
2,DISH,7.535,2.3,N/A
3,AAL,15.8,3.68,N/A
4,ZION,38.17,3.91,N/A
5,SIVB,106.04,4.18,N/A
6,VLO,129.79,4.46,N/A
7,GM,36.77,5.13,N/A
8,CMA,53.28,5.58,N/A
9,WU,11.895,6.04,N/A


## Calculating the Number of Shares to Buy
We now need to calculate the number of shares we need to buy. 

To do this, we will use the `portfolio_input` function that we created in our momentum project.

I have included this function below.

In [8]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the value of your portfolio:")

    try:
        val = float(portfolio_size)
    except ValueError:
        print("That's not a number! \n Try again:")
        portfolio_size = input("Enter the value of your portfolio:")

Use the `portfolio_input` function to accept a `portfolio_size` variable from the user of this script.

In [9]:
portfolio_input()

Enter the value of your portfolio:1000000


We can now use the global `portfolio_size` variable to calculate the number of shares that our strategy should purchase.

In [10]:
position_size = float(portfolio_size)/len(final_dataframe.index)
for row in final_dataframe.index:
    final_dataframe.loc[row, 'Number of Shares to Buy'] = math.floor(position_size/final_dataframe.loc[row, 'Price'])
    
final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,BRK.B,363.24,0.07,53
1,FRC,3.51,0.43,5586
2,DISH,7.535,2.3,2602
3,AAL,15.8,3.68,1241
4,ZION,38.17,3.91,513
5,SIVB,106.04,4.18,184
6,VLO,129.79,4.46,151
7,GM,36.77,5.13,533
8,CMA,53.28,5.58,368
9,WU,11.895,6.04,1648


## Building a Better (and More Realistic) Value Strategy
Every valuation metric has certain flaws.

For example, the price-to-earnings ratio doesn't work well with stocks with negative earnings.

Similarly, stocks that buyback their own shares are difficult to value using the price-to-book ratio.

Investors typically use a `composite` basket of valuation metrics to build robust quantitative value strategies. In this section, we will filter for stocks with the lowest percentiles on the following metrics:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* Enterprise Value divided by Earnings Before Interest, Taxes, Depreciation, and Amortization (EV/EBITDA)
* Enterprise Value divided by Gross Profit (EV/GP)

Some of these metrics aren't provided directly by the IEX Cloud API, and must be computed after pulling raw data. We'll start by calculating each data point from scratch.

In [11]:
symbol = 'AAPL'
batch_api_call_url = f'https://cloud.iexapis.com/stable/stock/market/batch/?types=advanced-stats,quote&symbols={symbol}&token={IEX_CLOUD_API_TOKEN}'
data = requests.get(batch_api_call_url).json()

# P/E Ratio
pe_ratio = data[symbol]['quote']['peRatio']

# P/B Ratio
pb_ratio = data[symbol]['advanced-stats']['priceToBook']

#P/S Ratio
ps_ratio = data[symbol]['advanced-stats']['priceToSales']

# EV/EBITDA
enterprise_value = data[symbol]['advanced-stats']['enterpriseValue']
ebitda = data[symbol]['advanced-stats']['EBITDA']
ev_to_ebitda = enterprise_value/ebitda

# EV/GP
gross_profit = data[symbol]['advanced-stats']['grossProfit']
ev_to_gross_profit = enterprise_value/gross_profit

Let's move on to building our DataFrame.

In [12]:
rv_columns = [
    'Ticker',
    'Price',
    'Number of Shares to Buy', 
    'Price-to-Earnings Ratio',
    'PE Percentile',
    'Price-to-Book Ratio',
    'PB Percentile',
    'Price-to-Sales Ratio',
    'PS Percentile',
    'EV/EBITDA',
    'EV/EBITDA Percentile',
    'EV/GP',
    'EV/GP Percentile',
    'RV Score'
]

rv_dataframe = pd.DataFrame(columns = rv_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://cloud.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote,advanced-stats&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        if symbol in data:
            enterprise_value = data[symbol]['advanced-stats']['enterpriseValue']
            ebitda = data[symbol]['advanced-stats']['EBITDA']
            gross_profit = data[symbol]['advanced-stats']['grossProfit']

            try:
                ev_to_ebitda = enterprise_value/ebitda
            except TypeError:
                ev_to_ebitda = np.NaN

            try:
                ev_to_gross_profit = enterprise_value/gross_profit
            except TypeError:
                ev_to_gross_profit = np.NaN

            rv_dataframe = rv_dataframe.append(
                pd.Series([
                    symbol,
                    data[symbol]['quote']['latestPrice'],
                    'N/A',
                    data[symbol]['quote']['peRatio'],
                    'N/A',
                    data[symbol]['advanced-stats']['priceToBook'],
                    'N/A',
                    data[symbol]['advanced-stats']['priceToSales'],
                    'N/A',
                    ev_to_ebitda,
                    'N/A',
                    ev_to_gross_profit,
                    'N/A',
                    'N/A'
            ],
            index = rv_columns),
                ignore_index = True
            )
        else:
            print(f'Symbol not found: {symbol}')

Symbol not found: DISCA
Symbol not found: HFC
Symbol not found: VIAC
Symbol not found: WLTW


In [13]:
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,126.275,N/A,27.81,N/A,6.45,N/A,5.3,N/A,19.608459,N/A,10.112924,N/A,N/A
1,AAL,15.81,N/A,3.69,N/A,-2.36,N/A,0.1957,N/A,4.760662,N/A,0.622646,N/A,N/A
2,AAP,70.66,N/A,10.5,N/A,1.6,N/A,0.3769,N/A,6.611546,N/A,1.196694,N/A,N/A
3,AAPL,178.395,N/A,30.34,N/A,46.05,N/A,7.43,N/A,23.558095,N/A,17.537101,N/A,N/A
4,ABBV,150.115,N/A,None,N/A,19.64,N/A,4.59,N/A,12.080336,N/A,8.021971,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
496,YUM,135.96,N/A,None,N/A,-4.29,N/A,5.42,N/A,20.996868,N/A,9.439293,N/A,N/A
497,ZBH,125.93,N/A,None,N/A,2.13,N/A,3.67,N/A,16.74002,N/A,6.159232,N/A,N/A
498,ZBRA,247.37,N/A,19.81,N/A,4.31,N/A,2.35,N/A,14.440478,N/A,5.875431,N/A,N/A
499,ZION,38.17,N/A,3.91,N/A,1.18,N/A,1.3,N/A,2.974565,N/A,1.36244,N/A,N/A


## Dealing With Missing Data in Our DataFrame

Our DataFrame contains some missing data./n
Identify missing data:

In [14]:
rv_dataframe[rv_dataframe.isnull().any(axis=1)]

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
4,ABBV,150.115,N/A,None,N/A,19.64,N/A,4.59,N/A,12.080336,N/A,8.021971,N/A,N/A
6,ABMD,381.02,N/A,65.69,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
7,ABT,107.98,N/A,None,N/A,5.04,N/A,4.65,N/A,24.159919,N/A,8.827788,N/A,N/A
18,AIG,62.28,N/A,None,N/A,1.03,N/A,0.8574,N/A,4.0914,N/A,0.857416,N/A,N/A
20,AIZ,143.375,N/A,None,N/A,1.68,N/A,0.7157,N/A,11.588329,N/A,0.715713,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
483,WMT,160.99,N/A,None,N/A,5.89,N/A,0.6855,N/A,14.333349,N/A,3.104572,N/A,N/A
491,XLNX,194.92,N/A,52.54,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
495,XYL,106.53,N/A,None,N/A,5.32,N/A,3.34,N/A,21.912776,N/A,9.289883,N/A,N/A
496,YUM,135.96,N/A,None,N/A,-4.29,N/A,5.42,N/A,20.996868,N/A,9.439293,N/A,N/A


We will eplace missing data with the average non-`NaN` data point from that column. 



In [15]:
for column in ['Price-to-Earnings Ratio', 'Price-to-Book Ratio','Price-to-Sales Ratio',  'EV/EBITDA','EV/GP']:
    rv_dataframe[column].fillna(rv_dataframe[column].mean(), inplace = True)

Now, if we run the statement from earlier to print rows that contain missing data, nothing should be returned:

In [16]:
rv_dataframe[rv_dataframe.isnull().any(axis=1)]

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score


## Calculating Value Percentiles

We now need to calculate value score percentiles for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* EV/EBITDA
* EV/GP

Here's how we'll do this:

In [17]:
metrics = {
            'Price-to-Earnings Ratio': 'PE Percentile',
            'Price-to-Book Ratio':'PB Percentile',
            'Price-to-Sales Ratio': 'PS Percentile',
            'EV/EBITDA':'EV/EBITDA Percentile',
            'EV/GP':'EV/GP Percentile'
}

for row in rv_dataframe.index:
    for metric in metrics.keys():
        rv_dataframe.loc[row, metrics[metric]] = stats.percentileofscore(rv_dataframe[metric], rv_dataframe.loc[row, metric])/100

for metric in metrics.values():
    print(rv_dataframe[metric])

rv_dataframe

0      0.768463
1      0.383234
2      0.446108
3      0.816367
4      0.218563
         ...   
496    0.218563
497    0.218563
498    0.654691
499     0.38523
500    0.898204
Name: PE Percentile, Length: 501, dtype: object
0      0.744511
1      0.065868
2      0.247505
3       0.98004
4      0.912176
         ...   
496    0.061876
497    0.356287
498    0.615768
499    0.148703
500    0.904192
Name: PB Percentile, Length: 501, dtype: object
0      0.799401
1      0.013972
2        0.0499
3      0.882236
4      0.750499
         ...   
496    0.804391
497    0.674651
498    0.482036
499    0.248503
500    0.944112
Name: PS Percentile, Length: 501, dtype: object
0      0.772455
1      0.073852
2      0.137725
3      0.860279
4      0.393214
         ...   
496    0.822355
497    0.666667
498    0.512974
499    0.023952
500    0.916168
Name: EV/EBITDA Percentile, Length: 501, dtype: object
0      0.742515
1      0.017964
2      0.065868
3      0.956088
4      0.590818
         ...   
4

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,126.275,N/A,27.810000,0.768463,6.45,0.744511,5.3000,0.799401,19.608459,0.772455,10.112924,0.742515,N/A
1,AAL,15.81,N/A,3.690000,0.383234,-2.36,0.065868,0.1957,0.013972,4.760662,0.073852,0.622646,0.017964,N/A
2,AAP,70.66,N/A,10.500000,0.446108,1.60,0.247505,0.3769,0.0499,6.611546,0.137725,1.196694,0.065868,N/A
3,AAPL,178.395,N/A,30.340000,0.816367,46.05,0.98004,7.4300,0.882236,23.558095,0.860279,17.537101,0.956088,N/A
4,ABBV,150.115,N/A,-1.415994,0.218563,19.64,0.912176,4.5900,0.750499,12.080336,0.393214,8.021971,0.590818,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
496,YUM,135.96,N/A,-1.415994,0.218563,-4.29,0.061876,5.4200,0.804391,20.996868,0.822355,9.439293,0.712575,N/A
497,ZBH,125.93,N/A,-1.415994,0.218563,2.13,0.356287,3.6700,0.674651,16.740020,0.666667,6.159232,0.437126,N/A
498,ZBRA,247.37,N/A,19.810000,0.654691,4.31,0.615768,2.3500,0.482036,14.440478,0.512974,5.875431,0.411178,N/A
499,ZION,38.17,N/A,3.910000,0.38523,1.18,0.148703,1.3000,0.248503,2.974565,0.023952,1.362440,0.095808,N/A


## Calculating the RV Score
We'll now calculate our RV Score (Robust Value)

The RV Score will be the arithmetic mean of the 4 percentile scores that we calculated in the last section.



In [18]:
from statistics import mean

for row in rv_dataframe.index:
    value_percentiles = []
    for metric in metrics.keys():
        value_percentiles.append(rv_dataframe.loc[row, metrics[metric]])
    rv_dataframe.loc[row, 'RV Score'] = mean(value_percentiles)
    
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,126.275,N/A,27.810000,0.768463,6.45,0.744511,5.3000,0.799401,19.608459,0.772455,10.112924,0.742515,0.765469
1,AAL,15.81,N/A,3.690000,0.383234,-2.36,0.065868,0.1957,0.013972,4.760662,0.073852,0.622646,0.017964,0.110978
2,AAP,70.66,N/A,10.500000,0.446108,1.60,0.247505,0.3769,0.0499,6.611546,0.137725,1.196694,0.065868,0.189421
3,AAPL,178.395,N/A,30.340000,0.816367,46.05,0.98004,7.4300,0.882236,23.558095,0.860279,17.537101,0.956088,0.899002
4,ABBV,150.115,N/A,-1.415994,0.218563,19.64,0.912176,4.5900,0.750499,12.080336,0.393214,8.021971,0.590818,0.573054
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
496,YUM,135.96,N/A,-1.415994,0.218563,-4.29,0.061876,5.4200,0.804391,20.996868,0.822355,9.439293,0.712575,0.523952
497,ZBH,125.93,N/A,-1.415994,0.218563,2.13,0.356287,3.6700,0.674651,16.740020,0.666667,6.159232,0.437126,0.470659
498,ZBRA,247.37,N/A,19.810000,0.654691,4.31,0.615768,2.3500,0.482036,14.440478,0.512974,5.875431,0.411178,0.535329
499,ZION,38.17,N/A,3.910000,0.38523,1.18,0.148703,1.3000,0.248503,2.974565,0.023952,1.362440,0.095808,0.180439


## Selecting the 50 Best Value Stocks¶

As before, we can identify the 50 best value stocks in our universe by sorting the DataFrame on the RV Score column and dropping all but the top 50 entries.

In [19]:
rv_dataframe.sort_values(by = 'RV Score', inplace = True)
rv_dataframe = rv_dataframe[:50]
rv_dataframe.reset_index(drop = True, inplace = True)

## Calculating the Number of Shares to Buy
We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a for loop to calculate the number of shares to buy for each stock in our investment universe.

In [20]:
portfolio_input()

Enter the value of your portfolio:1000000


In [21]:
position_size = float(portfolio_size) / len(rv_dataframe.index)
for i in range(0, len(rv_dataframe['Ticker'])-1):
    rv_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / rv_dataframe['Price'][i])
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,LNC,27.725,721,-1.560000,0.063872,0.9444,0.0998,0.2969,0.033932,-1.557325,0.015968,0.296918,0.003992,0.043513
1,XRX,15.47,1292,-9.100000,0.041916,0.7234,0.083832,0.3418,0.045908,5.301979,0.091816,0.979147,0.053892,0.063473
2,WBA,29.93,668,-7.790000,0.045908,1.2200,0.157685,0.1894,0.011976,-7.231873,0.013972,1.359240,0.093812,0.064671
3,KSS,28.86,693,-175.440000,0.00998,0.8651,0.093812,0.1792,0.00998,7.796019,0.179641,1.251622,0.071856,0.073054
4,FRC,3.51,5698,0.430000,0.379242,0.0456,0.067864,0.0934,0.001996,0.270279,0.017964,0.094878,0.001996,0.093812
5,GPS,10.63,1881,-67.110000,0.015968,1.7900,0.289421,0.2540,0.021956,6.891914,0.141717,0.836794,0.037924,0.101397
6,UA,7.16,2793,-1.415994,0.218563,1.4700,0.204591,0.5392,0.07485,3.074924,0.026946,0.719120,0.02495,0.10998
7,AAL,15.81,1265,3.690000,0.383234,-2.3600,0.065868,0.1957,0.013972,4.760662,0.073852,0.622646,0.017964,0.110978
8,UNM,50.98,392,-1.415994,0.218563,1.0600,0.125749,0.8125,0.123752,4.192791,0.055888,0.812472,0.035928,0.111976
9,L,63.71,313,-1.415994,0.218563,0.9716,0.103792,0.9351,0.147705,3.899149,0.045908,0.935148,0.047904,0.112774


## Formatting Our Excel Output



In [22]:
writer = pd.ExcelWriter('value_strategy.xlsx', engine='xlsxwriter')
rv_dataframe.to_excel(writer, sheet_name='Value Strategy', index = False)

## Creating the Formats We'll Need For Our .xlsx File


In [23]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

float_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [24]:
column_formats = {
                    'A': ['Ticker', string_template],
                    'B': ['Price', dollar_template],
                    'C': ['Number of Shares to Buy', integer_template],
                    'D': ['Price-to-Earnings Ratio', float_template],
                    'E': ['PE Percentile', percent_template],
                    'F': ['Price-to-Book Ratio', float_template],
                    'G': ['PB Percentile',percent_template],
                    'H': ['Price-to-Sales Ratio', float_template],
                    'I': ['PS Percentile', percent_template],
                    'J': ['EV/EBITDA', float_template],
                    'K': ['EV/EBITDA Percentile', percent_template],
                    'L': ['EV/GP', float_template],
                    'M': ['EV/GP Percentile', percent_template],
                    'N': ['RV Score', percent_template]
                 }

for column in column_formats.keys():
    writer.sheets['Value Strategy'].set_column(f'{column}:{column}', 25, column_formats[column][1])
    writer.sheets['Value Strategy'].write(f'{column}1', column_formats[column][0], column_formats[column][1])

## Saving Our Excel Output


In [25]:
writer.save()